## Inferencing a Finetuned Model

### Importing dependencies

In [1]:

!pip install python-dotenv
!pip install huggingface_hub

### Login to HuggingFace

In [2]:
from huggingface_hub import login
from google.colab import userdata

HF_token = userdata.get('HF_TOKEN')
login(token=HF_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### Load the Model from Hugging Face

In [3]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

username = "noelabu"
model_id = f"{username}/gemma-2b-instruct-ft-mental-health-conv_v2"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load the model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  # Adjust based on your saved model's dtype
    device_map="auto"  # Automatically maps model to available devices
)

tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/522 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

### Test the finetuned Model

In [4]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  user
  You are a mental health counselor engaging in a conversation with a client.
  The goal is to provide empathetic responses, offer support, and guide the client through their thoughts and feelings.
  Respond to the following client statement with a thoughtful and understanding reply.
  {query}
  \nmodel


  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return decoded


In [9]:
query = """I often find myself caught in a cycle of negative thoughts, feeling as though I don't matter to anyone.
          I wish I could see my worth, but instead, I focus on my flaws and shortcomings.
          I want to break free from this mindset and start believing that I can contribute positively to the world around me.
          What steps can I take to shift my perspective and cultivate self-acceptance?"""

result = get_completion(query=query, model=model, tokenizer=tokenizer)
print(result)


  user
  You are a mental health counselor engaging in a conversation with a client.
  The goal is to provide empathetic responses, offer support, and guide the client through their thoughts and feelings.
  Respond to the following client statement with a thoughtful and understanding reply.
  I often find myself caught in a cycle of negative thoughts, feeling as though I don't matter to anyone.
          I wish I could see my worth, but instead, I focus on my flaws and shortcomings.
          I want to break free from this mindset and start believing that I can contribute positively to the world around me.
          What steps can I take to shift my perspective and cultivate self-acceptance?
  
model


   You mentioned that your problem is that your own negative thoughts are blocking your ability to believe in your self-worth.   There are two different options for dealing with negative thoughts:  a) Trying to disprove that they will harm your self-worth & self-esteem by either proving